In [59]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

Pkg.add("CoordinateTransformations")
Pkg.add("RigidBodyDynamics")
Pkg.add("MeshCatMechanisms")
Pkg.add("MeshCat")

using CoordinateTransformations: Translation
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Printf
using LinearAlgebra: I

  Activating project at `~/cmu/courses/16745_OCRL/Project`
   Resolving package versions...
  No Changes to `~/cmu/courses/16745_OCRL/Project/Project.toml`
  No Changes to `~/cmu/courses/16745_OCRL/Project/Manifest.toml`
   Resolving package versions...
  No Changes to `~/cmu/courses/16745_OCRL/Project/Project.toml`
  No Changes to `~/cmu/courses/16745_OCRL/Project/Manifest.toml`
   Resolving package versions...
  No Changes to `~/cmu/courses/16745_OCRL/Project/Project.toml`
  No Changes to `~/cmu/courses/16745_OCRL/Project/Manifest.toml`
   Resolving package versions...
  No Changes to `~/cmu/courses/16745_OCRL/Project/Project.toml`
  No Changes to `~/cmu/courses/16745_OCRL/Project/Manifest.toml`


In [67]:
vis = Visualizer()
# open(vis)  # open the visualizer in a separate tab/window
render(vis) # render the visualizer here inside the jupyter notebook

┌ Info: Listening on: 127.0.0.1:8712, thread id: 1
└ @ HTTP.Servers /home/hayden/.julia/packages/HTTP/4AUPl/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8712
└ @ MeshCat /home/hayden/.julia/packages/MeshCat/9QrxD/src/visualizer.jl:43


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8712, HTTP.Servers.Server{HTTP.Servers.Listener{Nothing, Sockets.TCPServer}}(HTTP.Servers.Listener{Nothing, Sockets.TCPServer}(Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 8712), "127.0.0.1", "8712", nothing, Sockets.TCPServer(RawFD(76) active)), nothing, Set{HTTP.Connections.Connection}(), Task (runnable) @0x00007a739fbb2d60, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 134639305577360, 1))), MeshCat.AnimationContext[]))

In [68]:
urdf_path = "/home/hayden/cmu/courses/16745_OCRL/Project/zoe2.urdf"
robot = parse_urdf(urdf_path)
delete!(vis)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf_path), vis)

MechanismVisualizer{MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}, Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat at http://127.0.0.1:8712, 12)

In [ ]:
# Create a state for the mechanism
state = MechanismState(robot)
q = configuration(state)

println("Initial configuration vector:")
println(q)
println("Number of degrees of freedom: ", length(q))

# Iterate over the joints to print their names
for (i, joint) in enumerate(joints(robot))
    println("Joint ", i, ": ", joint.name)
end

# Example: update a joint value
# q[5] += 1.0

# Update the visualizer with the new configuration
set_configuration!(mvis, q)

Initial configuration vector:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Number of degrees of freedom: 7
Joint 1: axle_roll_back_joint
Joint 2: axle_yaw_front_joint
Joint 3: axle_yaw_back_joint
Joint 4: wheel_front_right_joint
Joint 5: wheel_front_left_joint
Joint 6: wheel_back_right_joint
Joint 7: wheel_back_left_joint


In [ ]:
# Xsim is a 

function animate_rover(Xsim, Xref, dt)

    # animate rover, show Xref with vis_traj!, and track Xref with the green sphere
    vis = Visualizer()
    robot_obj = parse_urdf(urdf_path)
    mc.setobject!(vis[:vic], robot_obj)

    target = mc.HyperSphere(mc.Point(0,0,0.0),0.1)
    mc.setobject!(vis[:target], target, mc.MeshPhongMaterial(color = mc.RGBA(0.0,1.0,0.0,0.4)))


    anim = mc.Animation(vis, fps=floor(Int,1/dt))
    for k = 1:length(Xsim)
        mc.atframe(anim, k) do
            r = Xsim[k][1:3]
            p = Xsim[k][7:9]
            mc.settransform!(vis[:vic], mc.compose(mc.Translation(r),mc.LinearMap(1.5*(dcm_from_mrp(p)))))
            mc.settransform!(vis[:target], mc.Translation(Xref[k][1:3]))
        end
    end
    mc.setanimation!(vis, anim)

    return (mc.render(vis))
end

In [64]:
# Parameters for the simulation
nsteps = 100       # total steps of the simulation
dt = 0.1           # time step (seconds)
forward_step = 0.05  # distance to move forward per step
wheel_speed_increment = 0.1  # amount to rotate each wheel per step

# We'll assume that joints 4, 5, 6, 7 correspond to the wheels.
# (See your printed joint names above.)
for k in 1:nsteps
    # Compute a new overall transform for the rover.
    # Here we move it forward along the x-axis.
    x = forward_step * k
    T = Translation([x, 0.0, 0.0])
    # Update the visualizer's transform. This applies a global offset.
    # (If mvis doesn't support a global transform update, you can update
    # the transform of the mechanism's group node via vis["mechanism"].)
    set_transform!(mvis, T)
    
    # Update the wheel joints to simulate spinning.
    q[4] += wheel_speed_increment
    q[5] += wheel_speed_increment
    q[6] += wheel_speed_increment
    q[7] += wheel_speed_increment
    set_configuration!(mvis, q)
    
    sleep(dt)  # pause for dt seconds to control the animation rate
end


UndefVarError: UndefVarError: `set_transform!` not defined